In [1]:
import torch
import sys
sys.executable
!{sys.executable} -m pip install kagglehub

In [ ]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

In [ ]:
model = Qwen3VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-2B-Instruct", dtype="auto", device_map="auto"
)

In [ ]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-2B-Instruct")  

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ledang14/vqa-test")

print("Path to dataset files:", path)

In [ ]:
from PIL import Image
import os
import pandas as pd

In [ ]:
for item in os.listdir(path):
    print(f"- {item}")

In [ ]:
image_folder = os.path.join(path, "custom_dataset/custom_dataset/train")
sample_image_path = os.path.join(image_folder, "fe5788c604751c347b0ba8f2f3566ad3.png")

In [ ]:
sample_image_path = os.path.join(image_folder, "e7160116758400f114d4153763c3e85d.png")
if os.path.exists(sample_image_path):
    image = Image.open(sample_image_path).convert("RGB")
    print(f"\nSuccessfully loaded image: {image.size} {image.format}")
    # img.show() # Note: This only works if a display server is available
else:
    print(f"\nCould not find a sample image at: {sample_image_path}")

In [ ]:
question =  "What number of matte blocks have the same color as the large matte cylinder?"

In [ ]:
 instructions_yes_no = (
        "Given an IMAGE and a QUESTION, answer in 'yes' or 'no', and give a concise explaination in this format:\n"
        "<answer> because <explanation>\n"
        f"Question: {question}"
    )

In [ ]:
 instructions_attributes = (
        "Given an IMAGE and a QUESTION, answer about these 4 attributes: shape (box, sphere, cylinder), color (gray, red, blue, green, brown, purple, cyan, yellow), size (large, small), or material (rubber, metallic), and give a concise explaination in this format:\n"
        "<answer> because <explanation>\n"
        f"Question: {question}"
    )

In [ ]:
instructions_counting = (
    "Given an IMAGE and a QUESTION, let’s first understand the problem and devise a plan to solve the problem. Then, let’s "
    "carry out the plan and solve the problem step by step. Answer one integer from 1 to 10, and give a concise explanation in this format:\n"
    "<answer> because <explanation>\n"
    f"Question: {question}" 
)

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {
                "type": "text",
                "text": instructions_counting,
            },
        ],
    },
]


In [ ]:
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
)
inputs = inputs.to(model.device)

In [ ]:

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

In [1]:
from test import *

In [2]:
from utils import *

In [4]:
root = "custom_dataset/custom_dataset"

train_labeled = load_custom_clevr(
    root=root,
    csv_path=os.path.join(root, "train_labels.csv"),
    require_image=True,
)

print(len(train_labeled))
# print(train_labeled[0])


0


In [39]:

@dataclass
class ClevrXExample:
    image_path: str
    question: str
    answer: Optional[str]          # None for unlabeled / test splits
    explanation: List[str]         # list of explanation sentences
    sample_id: str
    raw: Dict    

In [9]:
results: List[CLEVRXExample] = []

    # Candidate image dirs
img_dirs = [
    os.path.join(root, "train"),
    os.path.join(root, "test"),
    os.path.join(root, "train_non_labels"),
]




In [10]:
img_dirs

['custom_dataset/custom_dataset/train',
 'custom_dataset/custom_dataset/test',
 'custom_dataset/custom_dataset/train_non_labels']

In [16]:
# Load CSV
root = "custom_dataset/custom_dataset"
csv_path=os.path.join(root, "train_labels.csv")
with open(csv_path, "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    rows = list(reader)

In [17]:
rows

[{'id': '0',
  'file': 'fe5788c604751c347b0ba8f2f3566ad3.png',
  'question': 'Are there an equal number of cubes behind the tiny gray cube and brown metallic objects?',
  'answer': 'yes',
  'explanation': "['There is a cube which is behind the tiny gray cube and there is a brown metallic cube.', 'There is a cube which is behind the tiny gray cube and there is a brown metallic block.', 'There is a cube that is behind the tiny gray cube and there is a brown metallic cube.', 'There is a cube that is behind the tiny gray cube and there is a brown metallic block.']"},
 {'id': '1',
  'file': '440229924eabdfecfd6b5ea8924f110c.png',
  'question': 'There is a tiny red thing that is right of the brown cylinder behind the small rubber sphere; are there any tiny red balls on the right side of it?',
  'answer': 'no',
  'explanation': "['There are no tiny red balls which are on the right side of the tiny red cube that is right of the brown cylinder that is behind the small rubber sphere.', 'There ar

In [28]:

results: List[ClevrXExample] = []

In [86]:
import ast

with open(csv_path, "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    rows = list(reader)
    for row in rows:
        sample_id = str(row.get("id", ""))
        question = row.get("question", "")
        answer = row.get("answer") if "answer" in row else None
        file_name = row.get("file") or row.get("image") or row.get("filename")

        # -------- locate image file --------
        image_path: Optional[str] = None
        if file_name:
            for d in img_dirs:
                cand = os.path.join(d, file_name)
                if os.path.exists(cand):
                    image_path = cand
                    break

        # -------- parse explanation field --------
        expl_raw = row.get("explanation", "")    

        results.append(
            ClevrXExample(
                image_path=image_path or "",
                question=question,
                answer=answer,
                explanation=expl_raw,
                sample_id=sample_id,
                raw=row,
            )
        )


In [49]:
import ast

raw = "['There is a brown cylinder which has the identical color as a large shiny cylinder.', 'There is a brown cylinder that has the same color as a large shiny cylinder.']"

explanation_list = ast.literal_eval(raw)
print(explanation_list)

['There is a brown cylinder which has the identical color as a large shiny cylinder.', 'There is a brown cylinder that has the same color as a large shiny cylinder.']


In [1]:
from utils import *

In [12]:
from utils import load_custom_clevr, ClevrXExample, classify_clevr_question
from prompt_template import (prompt_binary_expl, prompt_counting_expl, prompt_attribute_expl)

In [46]:
root = "custom_dataset/custom_dataset"
csv_path = f"{root}/train_labels.csv"
csv_path = f"{root}/test_non_labels.csv"
data = load_custom_clevr(root, csv_path)

print(data[0].question)
print(data[0].explanation)
print(data[0].image_path)
# len(data)

What material is the yellow cylinder that is to the right of the tiny rubber object in front of the red metal block made of?
[]
custom_dataset/custom_dataset/test/cae1e431d2cab3622a5a8836776f7fed.png


In [14]:
from typing import List, Dict, Optional

In [26]:
def classify_question(answer: str) -> str:

    # 1. Binary questions have 'yes' or 'no' answers
    if answer.lower() in ['yes', 'no']:
        return 'binary'

    # 2. Counting questions have integer answers (0-10)
    # We check if the string can be converted to an integer and is in the range [0, 10]
    try:
        count = int(answer)
        if 0 <= count <= 10:
            return 'counting'
    except ValueError:
        # If it cannot be converted to an integer, it's an attribute
        pass

    # 3. Attribute questions have an answer corresponding to an attribute
    # (shape, color, size, or material)
    return 'attribute'

In [48]:
#training
def run_clevrx_task(
    dataset_root: str,
    csv_path: str,
    n_samples: Optional[int] = None,
    prompt_mode: str = "zero",
    training: bool = False,
):
    """
    Run CLEVR-X reasoning using Qwen3-VL on any CSV file
    (train_labels.csv, train_non_labels.csv, or test_non_labels.csv)
    
    - No split system.
    - No entropy.
    - Works with your simplified generate_answer().
    """

    # 1) Load dataset
    dataset = load_custom_clevr(dataset_root, csv_path)
    if n_samples is not None:
        dataset = dataset[:n_samples]

    print(f"Running CLEVR-X on {len(dataset)} samples (prompt_mode='{prompt_mode}')...\n")

    results = []
    total_correct = 0
    id_false_type_questions = []
    for i, s in enumerate(dataset, 1):
        question = s.question
        gt = s.answer  # may be None (unlabeled/test)

        # 2) Classify CLEVR-X question type
        qtype = classify_clevr_question(question)
        qtype_ground_truth = classify_question(gt)

        if qtype == qtype_ground_truth:
            total_correct += 1
        else:
            id_false_type_questions.append(s.sample_id)
        # 3) Build appropriate prompt
        if qtype == "binary":
            prompt = prompt_binary_expl(question, prompt_mode)
        elif qtype == "counting":
            prompt = prompt_counting_expl(question, prompt_mode)
        else:
            prompt = prompt_attribute_expl(question, prompt_mode)

        

        # 8) Store result
        results.append({
            "idx": i,
            "question": question,
            "ground_truth": gt,
            "image": s.image_path,
            "prompt_mode": prompt_mode,
            "qtype": qtype,
            "qtype_ground_truth": qtype_ground_truth,
        })

    print( f"Question Classification Accuracy: {total_correct}/{len(dataset)} = {total_correct / len(dataset):.3f}" )
    return results, id_false_type_questions

In [ ]:
#testing

def run_clevrx_task(
    dataset_root: str,
    csv_path: str,
    n_samples: Optional[int] = None,
    prompt_mode: str = "zero",
    training: bool = False,
):
    """
    Run CLEVR-X reasoning using Qwen3-VL on any CSV file
    (train_labels.csv, train_non_labels.csv, or test_non_labels.csv)
    
    - No split system.
    - No entropy.
    - Works with your simplified generate_answer().
    """

    # 1) Load dataset
    dataset = load_custom_clevr(dataset_root, csv_path)
    if n_samples is not None:
        dataset = dataset[:n_samples]

    print(f"Running CLEVR-X on {len(dataset)} samples (prompt_mode='{prompt_mode}')...\n")

    results = []
    total_correct = 0
    id_false_type_questions = []
    for i, s in enumerate(dataset, 1):
        question = s.question
        gt = s.answer  # may be None (unlabeled/test)

        # 2) Classify CLEVR-X question type
        qtype = classify_clevr_question(question)
        qtype_ground_truth = classify_question(gt)

        if qtype == qtype_ground_truth:
            total_correct += 1
        else:
            id_false_type_questions.append(s.sample_id)
        # 3) Build appropriate prompt
        if qtype == "binary":
            prompt = prompt_binary_expl(question, prompt_mode)
        elif qtype == "counting":
            prompt = prompt_counting_expl(question, prompt_mode)
        else:
            prompt = prompt_attribute_expl(question, prompt_mode)

        

        # 8) Store result
        results.append({
            "idx": i,
            "question": question,
            "ground_truth": gt,
            "image": s.image_path,
            "prompt_mode": prompt_mode,
            "qtype": qtype,
            "qtype_ground_truth": qtype_ground_truth,
        })

    print( f"Question Classification Accuracy: {total_correct}/{len(dataset)} = {total_correct / len(dataset):.3f}" )
    return results, id_false_type_questions

In [43]:
results, id_false_type_questions = run_clevrx_task(root, csv_path)

Running CLEVR-X on 5000 samples (prompt_mode='zero')...

Question Classification Accuracy: 5000/5000 = 1.000


In [44]:
id_false_type_questions

[]

In [40]:
results[7]

{'idx': 8,
 'question': 'What is the big red block made of?',
 'ground_truth': 'metal',
 'image': 'custom_dataset/custom_dataset/train/317a57651c937a366c6179263fbbd2d1.png',
 'prompt_mode': 'zero',
 'qtype': 'attibute',
 'qtype_ground_truth': 'attribute'}

In [1]:
from utils import *

In [2]:
root = "custom_dataset/custom_dataset"
csv_path = f"{root}/train_labels.csv"
csv_path = f"{root}/test_non_labels.csv"
data = load_custom_clevr(root, csv_path)

print(data[0].question)
print(data[0].image_path)
print(data[0].qtype)


What material is the yellow cylinder that is to the right of the tiny rubber object in front of the red metal block made of?
custom_dataset/custom_dataset/test/cae1e431d2cab3622a5a8836776f7fed.png
attribute


In [3]:
print(data[0])

ClevrXExample(image_path='custom_dataset/custom_dataset/test/cae1e431d2cab3622a5a8836776f7fed.png', question='What material is the yellow cylinder that is to the right of the tiny rubber object in front of the red metal block made of?', answer=None, explanation=[], sample_id='0', raw={'id': '0', 'file': 'cae1e431d2cab3622a5a8836776f7fed.png', 'question': 'What material is the yellow cylinder that is to the right of the tiny rubber object in front of the red metal block made of?'}, qtype='attribute')


In [4]:
import csv

def save_question_qtype_csv(
    dataset_root: str,
    input_csv_path: str,
    output_csv_path: str,
    training: bool = True,
):
    """
    Use load_custom_clevr() to load CLEVR-X data and save a CSV
    with only two columns: question and qtype.
    """
    # 1. Load dataset using your function
    dataset = load_custom_clevr(
        root=dataset_root,
        csv_path=input_csv_path,
        training=training,
    )

    # 2. Write out question + qtype
    with open(output_csv_path, "w", encoding="utf-8", newline="") as f_out:
        writer = csv.writer(f_out)
        writer.writerow(["question", "qtype"])  # header

        for ex in dataset:
            writer.writerow([ex.question, ex.qtype])

    print(f"Saved {len(dataset)} rows to {output_csv_path}")


In [1]:
dataset_root = "custom_dataset/custom_dataset"
input_csv  = f"{dataset_root}/test_non_labels.csv"
output_csv = f"{dataset_root}/test_question_types.csv"
csv_path = f"{dataset_root}/train_labels.csv"


In [2]:
from run import *

In [3]:
dataset = load_custom_clevr(dataset_root, csv_path, True)

In [4]:
len(dataset[0].explanation)

4

In [11]:
# dataset = dataset[]
result = []
img = Image.open("custom_dataset/custom_dataset/train/fe5788c604751c347b0ba8f2f3566ad3.png").convert("RGB")
for i, s in enumerate(dataset, 1):
        question = s.question
        gt = s.answer 
        image_path = s.image_path
        # img = Image.open(image_path).convert("RGB")
        qtype = classify_clevr_question(question)

        if qtype == "binary":
            prompt = prompt_binary_expl(img, question,3)
        elif qtype == "counting":
            prompt = prompt_counting_expl(img, question,3)
        else:
            prompt = prompt_attribute_expl(img, question,3)
        result.append({
            "idx": i,
            "question": question,
            "image": s.image_path,
            "prompt": prompt,
            "qtype": qtype,
        })

In [12]:
result

[{'idx': 1,
  'question': 'Are there an equal number of cubes behind the tiny gray cube and brown metallic objects?',
  'image': 'custom_dataset/custom_dataset/train/fe5788c604751c347b0ba8f2f3566ad3.png',
  'prompt': [{'role': 'system',
    'content': [{'type': 'text',
      'text': "You are a visual reasoning assistant for a Vision Questioning Answer dataset. \n                Your task is to analyze the provided image and answer binary (yes/no) questions based on that image of synthetic scenes. \n                Each image contains simple 3D objects (cubes, spheres, cylinders) with different colors, sizes, and materials. \n                Internally, break the problem into subtasks, plan your reasoning, and perform all necessary visual checks. \n                However, **do not show your reasoning steps in your response**. Your final output must strictly follow this format:\n\n\n                <answer> because <explanation>\n\n\n                Guidelines:\n\n                - <ans